**Library Imports and Initial Setup**

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from google.colab import drive
from sklearn.metrics import classification_report

For the purpose of this exercise we used the Italian language from the Universal Dependencies treebanks. Specifically we used the Venice Italian Treebank (https://github.com/UniversalDependencies/UD_Italian-VIT/tree/master).

**Preprocessing - Data Extraction**

We read files in CoNLL-U format (common for linguistic data annotations) and extract sentences and their word annotations. The function also ensures that sentences are properly formatted by wrapping them with markers (< s > for start and < e > for end).

In [ ]:
# Function to add space after commas and periods if they are attached directly to words
def add_space(sentence):
    # Using regular expression to find comma or period attached to a word
    sentence = re.sub(r'(\w)([,.])', r'\1 \2', sentence)
    return sentence

# Function to read CoNLL-U formatted files and extract sentences and word annotations
def read_conllu(file_path):
    sentences = []
    words = []
    POS_tags = []
    index = []
    sentence = None

    # Open the file and process it line by line
    with open(file_path, 'r', encoding='utf-8') as file:
        i = -1
        for line in file:
            line = line.strip()
            if line == '':
                if sentence:
                    sentences.append(add_space(sentence))
                    sentence = None
            elif line.startswith('# text'):
                i += 1
                sentence = "<s>" + line.split("=")[1] + " <e>"
            elif line.startswith('#'):
                continue
            else:
                tokens = line.split('\t')
                words.append(tokens[1])
                index.append(i)
                POS_tags.append(tokens[3])
        sent_df = pd.DataFrame({'Sentences': sentences})
        words_df = pd.DataFrame({'Words': words,'Sentence': index, 'POS': POS_tags})
    return sent_df, words_df

In [ ]:
# Connect to Google Drive
drive.mount('/content/drive', force_remount=True)
# Set path for training data
file_path_train = '/content/drive/My Drive/Colab Notebooks/it_vit-ud-train.conllu'
train_sent_df, train_words_df = read_conllu(file_path_train)
# Set paths for testing data
file_path_test = '/content/drive/My Drive/Colab Notebooks/it_vit-ud-test.conllu'
test_sent_df, test_words_df = read_conllu(file_path_test)
# Set path for dev data
file_path_dev = '/content/drive/My Drive/Colab Notebooks/it_vit-ud-dev.conllu'
dev_sent_df, dev_words_df = read_conllu(file_path_dev)

print("Train set Sentences: \n")
print(train_sent_df,'\n\n')
print("Train set Words: \n")
print(train_words_df,'\n\n')

print("Test set Sentences: \n")
print(test_sent_df,'\n\n')
print("Test set Words: \n")
print(test_words_df,'\n\n')

print("Dev set Sentences: \n")
print(dev_sent_df,'\n\n')
print("Dev set Words: \n")
print(dev_words_df,'\n\n')

Mounted at /content/drive
Train set Sentences: 

                                              Sentences
0     <s> Le infrastrutture come fattore di competit...
1     <s> Negli ultimi anni la dinamica dei polo di ...
2     <s> Il raggiungimento e il mantenimento di pos...
3     <s> Quest'ultimo è funzione di variabili strut...
4     <s> Il contesto milanese , se da un lato è sta...
...                                                 ...
8272                      <s> Premio Elsa Morante . <e>
8273  <s> È nato il premio Elsa Morante che verrà as...
8274  <s> Questo Premio che non avrà sede fissa né s...
8275  <s> sono promotori dell'iniziativa Patrizia Ca...
8276                           <s> Libri in campo . <e>

[8277 rows x 1 columns] 


Train set Words: 

                 Words  Sentence    POS
0                   Le         0    DET
1       infrastrutture         0   NOUN
2                 come         0    ADP
3              fattore         0   NOUN
4                   di         0

In [ ]:
print("Average Sentence Length (train set): ",train_sent_df['Sentences'].apply(lambda x: len(x.split())).mean())
print("Number of Unique Words (train set): ",len(train_words_df["Words"].unique()),"\n")

print("Average Sentence Length (test set): ",test_sent_df['Sentences'].apply(lambda x: len(x.split())).mean())
print("Number of Unique Words (test set): ",len(test_words_df["Words"].unique()),"\n")

print("Average Sentence Length (dev set): ",dev_sent_df['Sentences'].apply(lambda x: len(x.split())).mean())
print("Number of Unique Words (dev set): ",len(dev_words_df["Words"].unique()),"\n")

Average Sentence Length (train set):  25.73082034553582
Number of Unique Words (train set):  23045 

Average Sentence Length (test set):  23.529522024367385
Number of Unique Words (test set):  5850 

Average Sentence Length (dev set):  35.51682368775236
Number of Unique Words (dev set):  3637 



In [ ]:
print('Train Set POS Tags: \n',train_words_df['POS'].unique(),'\n')
print('Number of Unique POS Tags (Train): ',len(train_words_df['POS'].unique()),'\n\n')
print('Test Set POS Tags: \n',test_words_df['POS'].unique(),'\n')
print('Number of Unique POS Tags (Test): ',len(test_words_df['POS'].unique()),'\n\n')
print('Dev Set POS Tags: \n',dev_words_df['POS'].unique(),'\n')
print('Number of Unique POS Tags (Dev): ',len(dev_words_df['POS'].unique()))

Train Set POS Tags: 
 ['DET' 'NOUN' 'ADP' 'PROPN' 'PUNCT' '_' 'ADJ' 'AUX' 'ADV' 'VERB' 'PRON'
 'CCONJ' 'SCONJ' 'NUM' 'SYM' 'X' 'INTJ' 'PART'] 

Number of Unique POS Tags (Train):  18 


Test Set POS Tags: 
 ['ADV' 'AUX' 'VERB' 'NOUN' 'ADJ' '_' 'ADP' 'DET' 'PUNCT' 'CCONJ' 'PRON'
 'NUM' 'SCONJ' 'PROPN' 'X' 'INTJ' 'SYM'] 

Number of Unique POS Tags (Test):  17 


Dev Set POS Tags: 
 ['VERB' 'DET' 'NOUN' 'ADJ' 'PUNCT' 'ADV' 'CCONJ' 'PRON' 'ADP' 'AUX' 'NUM'
 'PROPN' '_' 'X' 'SCONJ' 'INTJ'] 

Number of Unique POS Tags (Dev):  16


**Data Processing and DataFrame (Windows) Creation**

We create new dataframes that represent "windows" around each word, capturing the local context of words. These dataframes are then saved as CSV files.

In [ ]:
from itertools import combinations
from tqdm import tqdm

# Function to generate three-word combinations from a sentence
def generate_three_word_combinations(sentence):
    combinations = []
    words = sentence.split()
    for i in range(0,len(words)):
        if (words[i] == "<s>") or (words[i] == "<e>"):
            continue
        combinations.append(list([words[i-1], words[i], words[i+1]]))
    return combinations

# Function to generate a dataframe from sentences and a word dataframe
def generate_df(sent_df ,word_df,set_name):
    new_data = {'Wi-1': [], 'Wi': [], 'Wi+1': [], "Wi_POS_tag" : []}
    i = 0
    for sentence in tqdm(sent_df['Sentences'], desc="Processing" + set_name + "Sentences"):
        combinations_list = generate_three_word_combinations(sentence)
        for combination in combinations_list:
            if (not word_df[(word_df["Words"]==combination[1]) & (word_df["Sentence"]==i)]['POS'].empty):
                new_data['Wi-1'].append(combination[0])
                new_data['Wi'].append(combination[1])
                new_data['Wi+1'].append(combination[2])
                new_data["Wi_POS_tag"].append(word_df[(word_df["Words"]==combination[1]) & (word_df["Sentence"]==i)]['POS'].values[0])
        i += 1
    windows_df = pd.DataFrame(new_data)
    return windows_df



In [ ]:
####################################################################
##  The following code uses the generate_df function on the data  ##
##  to create new dataframes in a convenient form                 ##
##  to use later during the training of our model.                ##
##  We then save the dataframes as .csv files.                    ##
####################################################################

#train_windows_df = generate_df(train_sent_df, train_words_df, "Train")
#test_windows_df = generate_df(test_sent_df, test_words_df, "Test")
#dev_windows_df = generate_df(dev_sent_df, dev_words_df, "Development")

#train_windows_df.to_csv("/content/drive/My Drive/Colab Notebooks/train_windows.csv", index=False)
#test_windows_df.to_csv("/content/drive/My Drive/Colab Notebooks/test_windows.csv", index=False)
#dev_windows_df.to_csv("/content/drive/My Drive/Colab Notebooks/dev_windows.csv", index=False)

**Baseline Model for POS Tagging**

We have implememented a simple majority voting baseline model for POS tagging, where the most frequent POS tag for a word in the training set is used as the prediction for that word in the testing set. The function then evaluates the model using a classification report from sklearn.

In [ ]:
# Function to implement a majority voting baseline for POS tagging
def majority_baseline(train_df, test_df):
    most_frequent_tags = train_df.groupby('Words')['POS'].agg(lambda x: x.mode().iloc[0]).to_dict()
    test_df['prediction'] = test_df['Words'].map(most_frequent_tags)
    most_frequent_tag_training = train_df['POS'].mode().iloc[0]
    test_df['prediction'].fillna(most_frequent_tag_training, inplace=True)
    report = classification_report(test_df['POS'], test_df['prediction'])
    print(report)

majority_baseline(train_words_df, test_words_df)

              precision    recall  f1-score   support

         ADJ       0.82      0.68      0.74      1565
         ADP       0.99      0.99      0.99      3857
         ADV       0.92      0.92      0.92      1258
         AUX       0.86      0.97      0.91      1059
       CCONJ       0.98      0.92      0.95       762
         DET       0.94      0.98      0.96      3902
        INTJ       0.69      0.69      0.69        16
        NOUN       0.72      0.95      0.82      4962
         NUM       0.98      0.86      0.91       418
        PRON       0.79      0.78      0.79      1256
       PROPN       0.98      0.55      0.71      1290
       PUNCT       1.00      1.00      1.00      3351
       SCONJ       0.84      0.40      0.54       320
         SYM       1.00      1.00      1.00         3
        VERB       0.93      0.63      0.75      2280
           X       0.67      0.26      0.38        38
           _       0.99      0.96      0.98      1634

    accuracy              